In [1]:
!pip install lightgbm

  Obtaining dependency information for lightgbm from https://files.pythonhosted.org/packages/d9/28/3be76b591a2e14a031b681b8283acf1dec2ad521f6f1701b7957df68c466/lightgbm-4.5.0-py3-none-win_amd64.whl.metadata
   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
    --------------------------------------- 0.0/1.4 MB 325.1 kB/s eta 0:00:05
   - -------------------------------------- 0.0/1.4 MB 326.8 kB/s eta 0:00:05
   -- ------------------------------------- 0.1/1.4 MB 581.0 kB/s eta 0:00:03
   --- ------------------------------------ 0.1/1.4 MB 708.1 kB/s eta 0:00:02
   ------ --------------------------------- 0.2/1.4 MB 901.1 kB/s eta 0:00:02
   --------- ------------------------------ 0.3/1.4 MB 1.2 MB/s eta 0:00:01
   ------------- -------------------------- 0.5/1.4 MB 1.3 MB/s eta 0:00:01
   ----------------- ---------------------- 0.6/1.4 MB 1.6 MB/s eta 0:00:01
   ---------------------- -----------

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
import lightgbm as lgb
from sklearn.metrics import accuracy_score
import numpy as np
import joblib

In [3]:
train_dir = r'D:\\my project\\fourth year\\computer vision\\dataset\\train'  
test_dir = r'D:\\my project\\fourth year\\computer vision\\dataset\\test'

In [4]:
# Set the input shape
input_shape = (128, 128, 3)
batch_size = 32

In [5]:
# Use ImageDataGenerator for data augmentation
data_generator = ImageDataGenerator(rescale=1./255)

In [6]:
# Load the training images
train_generator = data_generator.flow_from_directory(
    train_dir,
    target_size=(input_shape[0], input_shape[1]),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

Found 4010 images belonging to 2 classes.


In [7]:
# Load the test images
test_generator = data_generator.flow_from_directory(
    test_dir,
    target_size=(input_shape[0], input_shape[1]),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

Found 3543 images belonging to 2 classes.


In [8]:
# Define the model architecture up to the feature extraction layer
model = Sequential([
    Conv2D(32, (3, 3), input_shape=input_shape),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3)),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(64),
    Activation('relu'),
    Dropout(0.5)
])

C:\Users\mohamed\anaconda\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [9]:
# Extract features from training images 
train_features = model.predict(train_generator)
train_features = train_features.reshape(train_features.shape[0], -1)  # Flatten features
train_labels = train_generator.classes

C:\Users\mohamed\anaconda\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


126/126 ━━━━━━━━━━━━━━━━━━━━ 9s 73ms/step


In [10]:
# Extract features from test images 
test_features = model.predict(test_generator)
test_features = test_features.reshape(test_features.shape[0], -1)
test_labels = test_generator.classes

 34/111 ━━━━━━━━━━━━━━━━━━━━ 5s 69ms/step

C:\Users\mohamed\anaconda\Lib\site-packages\PIL\Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


111/111 ━━━━━━━━━━━━━━━━━━━━ 15s 132ms/step


In [12]:
# Use LightGBM 
lgb_classifier = lgb.LGBMClassifier(
    boosting_type='gbdt',  # Use gradient boosting
    num_leaves=31,         # Maximum leaves in one tree
    max_depth=-1,          # No maximum depth
    learning_rate=0.1,     # Step size
    n_estimators=100,      # Number of trees
    objective='binary',    # multi classification
)
lgb_classifier.fit(train_features, train_labels)

[LightGBM] [Info] Number of positive: 2010, number of negative: 2000
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000867 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 13439
[LightGBM] [Info] Number of data points in the train set: 4010, number of used features: 60
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.501247 -> initscore=0.004988
[LightGBM] [Info] Start training from score 0.004988


LGBMClassifier(objective='binary')

In [13]:
# Make predictions on the test data
test_predictions = lgb_classifier.predict(test_features)

In [14]:
# Calculate accuracy
test_accuracy = accuracy_score(test_labels, test_predictions)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

Test Accuracy: 63.53%


In [15]:
# Save the Keras CNN model as an .h5 file
model.save('cnn_feature_extractor8.h5')

In [16]:
# Save the trained LightGBM model
joblib.dump(lgb_classifier, 'lgb_classifier.pkl')

['lgb_classifier.pkl']

In [17]:
from keras.models import load_model
import joblib
import numpy as np
from tensorflow.keras.preprocessing import image

# Load the saved models
cnn_model = load_model('cnn_feature_extractor8.h5')
dt_classifier = joblib.load('lgb_classifier.pkl')

def classify_image(img_path):
    # Load and preprocess the new image
    img = image.load_img(img_path, target_size=(128, 128))  # Resize to match the CNN input
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.0  # Normalize to match the training preprocessing

    # Extract features using the CNN model
    features = cnn_model.predict(img_array)
    features = features.reshape(1, -1)  # Flatten to 1D for lightgbm

    # Classify the features with the lightgbm model
    prediction = dt_classifier.predict(features)
    
    # Interpret the prediction
    class_labels = {0: 'with_mask', 1: 'without_mask'}  # Adjust based on your dataset labels
    result = class_labels[prediction[0]]
    return result

# Test the function with a new image
img_path = r'D:\my project\fourth year\computer vision\dataset\test\with_mask\with_mask_2019.jpg'
result = classify_image(img_path)
print(f"The image is classified as: {result}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
The image is classified as: with_mask
